# 社交媒体行业级分析（中文教学版 | A+B+C 全覆盖 | 含一键安装与统一配置）


## Chunk 0a — 一键安装依赖（可选运行）
**作用**：一键安装/升级关键依赖。已安装可跳过。重启内核后再从头运行。


In [10]:
# === Chunk 0a: 一键安装依赖（可选） ===
%pip install -q --upgrade pip setuptools wheel

# 主流建模 & 解释 & 时间序列
%pip install -q \
  numpy pandas matplotlib scikit-learn \
  catboost==1.2.5 xgboost==2.1.1 lightgbm==4.5.0 \
  shap==0.46.0 statsmodels==0.14.2 pmdarima==2.0.4

# Prophet（如需使用，可能需要后端 CmdStan，见下方注释）
%pip install -q prophet==1.1.5 cmdstanpy==1.2.4

print("✅ 安装完成：若首次安装/升级了核心包，建议重启内核（Kernel → Restart Kernel）后再运行后续单元。")

# ===== 可选：Prophet 后端（CmdStan）一键安装 =====
# 某些环境下 Prophet 需要先编译 CmdStan。联网耗时较长，按需开启。
# 运行后建议重启内核。
# from cmdstanpy import install_cmdstan
# install_cmdstan()  # 如网络/权限受限，可跳过；Chunk 7 会自动降级到其他方法

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ 安装完成：若首次安装/升级了核心包，建议重启内核（Kernel → Restart Kernel）后再运行后续单元。


## Chunk 0b — 统一配置区
**作用**：集中管理随机种子、切分比例、爆款阈值、候选模型列表与通用超参。后续单元自动引用。


In [1]:
# === Chunk 0b: 统一配置 ===
CFG = {
    # 基础
    "csv_path": "Cleaned_Viral_Social_Media_Trends.csv",
    "date_col": "Post_Date",
    "random_state": 42,
    "test_size": 0.25,

    # 爆款阈值（平台内 Top p% 视为爆款）
    "viral_percentile": 0.90,   # 改成 0.95 = 前 5% 视为爆款

    # 候选模型优先级（从左到右尝试，自动回退）
    "reg_backends": ["catboost", "xgboost", "lightgbm", "rf"],
    "clf_backends": ["catboost", "xgboost", "lightgbm", "rf"],

    # 通用超参（按需调整）
    "catboost": {"depth": 6, "learning_rate": 0.08, "iterations": 600, "verbose": 0},
    "xgboost":  {"n_estimators": 600, "learning_rate": 0.05, "max_depth": 6, "subsample": 0.9, "colsample_bytree": 0.9},
    "lightgbm": {"n_estimators": 700, "learning_rate": 0.05, "num_leaves": 31},
    "rf":       {"n_estimators": 400},

    # 可选开关（遇到编译/权限问题时很有用）
    "force_disable_prophet": False,  # True 时强制关闭 Prophet，走降级路线
    "max_shap_samples": 300,         # SHAP 取样数量（控制速度）
}

print("✅ 已载入统一配置 CFG：")
for k, v in CFG.items():
    print(f"  {k}: {{...}}" if isinstance(v, dict) else f"  {k}: {v}")

✅ 已载入统一配置 CFG：
  csv_path: Cleaned_Viral_Social_Media_Trends.csv
  date_col: Post_Date
  random_state: 42
  test_size: 0.25
  viral_percentile: 0.9
  reg_backends: ['catboost', 'xgboost', 'lightgbm', 'rf']
  clf_backends: ['catboost', 'xgboost', 'lightgbm', 'rf']
  catboost: {...}
  xgboost: {...}
  lightgbm: {...}
  rf: {...}
  force_disable_prophet: False
  max_shap_samples: 300


## Chunk 1 — 导入依赖与环境检测


In [2]:
# === Chunk 1: 导入依赖与环境检测 ===
import os, sys, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    classification_report, confusion_matrix, silhouette_score
)

# 建模框架探测
try:
    from catboost import CatBoostRegressor, CatBoostClassifier
    CATBOOST_OK = True
except Exception:
    CATBOOST_OK = False

try:
    from xgboost import XGBRegressor, XGBClassifier
    XGBOOST_OK = True
except Exception:
    XGBOOST_OK = False

try:
    from lightgbm import LGBMRegressor, LGBMClassifier
    LGBM_OK = True
except Exception:
    LGBM_OK = False

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.cluster import KMeans

# 解释性
try:
    import shap
    SHAP_OK = True
except Exception:
    SHAP_OK = False

# 时间序列（Prophet 可选）
PROPHET_OK = False
try:
    from prophet import Prophet
    PROPHET_OK = True
except Exception:
    try:
        from fbprophet import Prophet  # 兼容老包名
        PROPHET_OK = True
    except Exception:
        PROPHET_OK = False

# statsmodels 季节分解（降级方案）
try:
    from statsmodels.tsa.seasonal import seasonal_decompose
    SM_SEASONAL_OK = True
except Exception:
    SM_SEASONAL_OK = False

# 统一配置对 Prophet 的硬开关（遇到编译/权限问题时开启 True）
if CFG.get("force_disable_prophet", False):
    PROPHET_OK = False

print("✅ 环境检测：")
print("  CatBoost:", "可用" if CATBOOST_OK else "不可用")
print("  XGBoost:", "可用" if XGBOOST_OK else "不可用")
print("  LightGBM:", "可用" if LGBM_OK else "不可用")
print("  SHAP:", "可用" if SHAP_OK else "不可用")
print("  Prophet:", "可用" if PROPHET_OK else "不可用", "| seasonal_decompose:", "可用" if SM_SEASONAL_OK else "不可用")

# 小提示：若 Prophet=不可用 或运行报错，可在 CFG 中把 force_disable_prophet 设为 True，
# 后续时间序列 Chunk 会自动降级到 seasonal_decompose / Auto-ARIMA / 移动平均。


✅ 环境检测：
  CatBoost: 可用
  XGBoost: 可用
  LightGBM: 可用
  SHAP: 可用
  Prophet: 可用 | seasonal_decompose: 可用


## Chunk 2 — 读取数据与基础清洗


In [3]:
# === Chunk 2: Load & basic clean ===
CSV_PATH = CFG["csv_path"]
if not os.path.exists(CSV_PATH):
    print("❌ 未找到 CSV：", CSV_PATH)
else:
    df = pd.read_csv(CSV_PATH)
    print("数据行数/列数：", df.shape)
    # 时间
    dtcol = CFG["date_col"]
    if dtcol in df.columns:
        df[dtcol] = pd.to_datetime(df[dtcol], errors="coerce")
        df["Year"] = df[dtcol].dt.year
        df["Month"] = df[dtcol].dt.month
        df["Weekday"] = df[dtcol].dt.dayofweek
        df["Hour"] = df[dtcol].dt.hour
    # 互动率
    for c in ["Views","Likes","Shares","Comments"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    if "Engagement_Rate" not in df.columns and all(c in df.columns for c in ["Views","Likes","Shares","Comments"]):
        df["Engagement_Rate"] = (df["Likes"].fillna(0)+df["Shares"].fillna(0)+df["Comments"].fillna(0)) / df["Views"].replace(0, np.nan)
        df["Engagement_Rate"] = df["Engagement_Rate"].fillna(0.0)
    print("列清单：", list(df.columns)[:20], "...")
    display(df.head(3))

数据行数/列数： (5000, 11)
列清单： ['Post_ID', 'Post_Date', 'Platform', 'Hashtag', 'Content_Type', 'Region', 'Views', 'Likes', 'Shares', 'Comments', 'Engagement_Level', 'Year', 'Month', 'Weekday', 'Hour', 'Engagement_Rate'] ...


,Post_ID,Post_Date,Platform,Hashtag,Content_Type,Region,Views,Likes,Shares,Comments,Engagement_Level,Year,Month,Weekday,Hour,Engagement_Rate
0,Post_1,2022-01-13,TikTok,#Challenge,Video,UK,4163464,339431,53135,19346,High,2022,1,3,0,0.098935
1,Post_2,2022-05-13,Instagram,#Education,Shorts,India,4155940,215240,65860,27239,Medium,2022,5,4,0,0.074192
2,Post_3,2022-01-07,Twitter,#Challenge,Video,Brazil,3666211,327143,39423,36223,Medium,2022,1,4,0,0.109865


## Chunk 3 — 快速 EDA 与平台对比


In [4]:
# === Chunk 3: EDA platform compare ===
if 'Platform' in df.columns:
    metrics = [c for c in ["Views","Likes","Shares","Comments","Engagement_Rate"] if c in df.columns]
    if metrics:
        eda = df.groupby("Platform")[metrics].mean().round(4).sort_values(metrics[0], ascending=False)
        print("各平台平均指标：")
        display(eda)
        print("分析：平台均值反映规模差异，不等同ROI；后续建模将进行归因与预测。")
    else:
        print("⚠️ 无可用数值指标用于平台对比。")
else:
    print("⚠️ 缺少 Platform 列，跳过平台对比。")

各平台平均指标：


,Views,Likes,Shares,Comments,Engagement_Rate
Platform,,,,,
YouTube,2.545648e+06,258314.0023,50073.0914,25321.3905,0.5064
TikTok,2.515015e+06,244206.7198,51468.2563,24778.6968,0.6387
Twitter,2.506005e+06,245880.1188,50227.7508,24456.8571,0.4654
Instagram,2.404080e+06,257118.2178,50310.9092,24958.1139,0.6621


分析：平台均值反映规模差异，不等同ROI；后续建模将进行归因与预测。


## Chunk 4 — 特征/目标设定（回归：预测互动率）


In [5]:
# === Chunk 4: Feature/Target for Regression ===
target_reg = "Engagement_Rate" if "Engagement_Rate" in df.columns else None
cat_cols = [c for c in ["Platform","Content_Type","Region","Hashtag"] if c in df.columns]
num_cols = [c for c in ["Views","Likes","Shares","Comments","Hour","Weekday","Month"] if c in df.columns]

if target_reg is None:
    print("❌ 没有 Engagement_Rate，无法做回归预测。")
else:
    if "Hashtag" in cat_cols:
        freq = df["Hashtag"].astype(str).value_counts()
        top = set(freq.head(50).index)
        df["Hashtag"] = df["Hashtag"].astype(str).where(df["Hashtag"].isin(top), "_other_")

    X_reg = df[cat_cols + num_cols].copy()
    y_reg = df[target_reg].astype(float).values

    preprocessor = ColumnTransformer([
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", StandardScaler(), num_cols),
    ])
    Xtr, Xte, ytr, yte = train_test_split(X_reg, y_reg, test_size=CFG["test_size"], random_state=CFG["random_state"])
    print("回归任务 — X 维度：", X_reg.shape, "  y 样本：", len(y_reg))
    print("类别特征：", cat_cols)
    print("数值特征：", num_cols)

回归任务 — X 维度： (5000, 11)   y 样本： 5000
类别特征： ['Platform', 'Content_Type', 'Region', 'Hashtag']
数值特征： ['Views', 'Likes', 'Shares', 'Comments', 'Hour', 'Weekday', 'Month']


## Chunk 5 — 回归模型对比（遵循统一配置 CFG）


In [6]:
# === Chunk 5: Regression model comparison ===
def build_reg_model(backend: str):
    if backend == "catboost" and CATBOOST_OK:
        return ("CatBoostRegressor", CatBoostRegressor(random_state=CFG["random_state"], **CFG["catboost"]))
    if backend == "xgboost" and XGBOOST_OK:
        return ("XGBRegressor", XGBRegressor(random_state=CFG["random_state"], **CFG["xgboost"], objective="reg:squarederror"))
    if backend == "lightgbm" and LGBM_OK:
        return ("LGBMRegressor", LGBMRegressor(random_state=CFG["random_state"], **CFG["lightgbm"]))
    if backend == "rf":
        return ("RandomForestRegressor", RandomForestRegressor(random_state=CFG["random_state"], n_jobs=-1, **CFG["rf"]))
    return None

reg_models = []
for b in CFG["reg_backends"]:
    pair = build_reg_model(b)
    if pair is not None: reg_models.append(pair)

reg_results = []
if target_reg is not None:
    for name, mdl in reg_models:
        pipe = Pipeline([("prep", preprocessor), ("mdl", mdl)])
        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)
        rmse = mean_squared_error(yte, pred, squared=False)
        mae  = mean_absolute_error(yte, pred)
        r2   = r2_score(yte, pred)

        reg_results.append((name, rmse, mae, r2, pipe))
        print(f"📊 模型：{name}")
        print(f"  RMSE: {rmse:.6f}（越小越好）")
        print(f"  MAE:  {mae:.6f}（越小越好）")
        print(f"  R²:   {r2:.4f}（越接近 1 越好）")
        print("-"*60)

    reg_df = pd.DataFrame(reg_results, columns=["model","RMSE","MAE","R2","pipe"]).sort_values("RMSE")
    display(reg_df[["model","RMSE","MAE","R2"]])
    best_reg = reg_df.iloc[0]
    print(f"✅ 最佳回归模型：{best_reg['model']} | RMSE={best_reg['RMSE']:.6f}, R²={best_reg['R2']:.4f}")
else:
    print("⚠️ 无法进行回归对比。")

📊 模型：CatBoostRegressor
  RMSE: 7.263349（越小越好）
  MAE:  0.309797（越小越好）
  R²:   0.2822（越接近 1 越好）
------------------------------------------------------------
📊 模型：XGBRegressor
  RMSE: 7.102914（越小越好）
  MAE:  0.332713（越小越好）
  R²:   0.3135（越接近 1 越好）
------------------------------------------------------------
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000323 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1097
[LightGBM] [Info] Number of data points in the train set: 3750, number of used features: 34
[LightGBM] [Info] Start training from score 0.488445
📊 模型：LGBMRegressor
  RMSE: 7.394393（越小越好）
  MAE:  0.419739（越小越好）
  R²:   0.2560（越接近 1 越好）
------------------------------------------------------------
📊 模型：RandomForestRegressor
  RMSE: 6.413075（越小越好）
  MAE:  0.257073（越小越好）
  R²:   0.4404（越接近 1 越好）
------------------------------------------------------------


,model,RMSE,MAE,R2
3,RandomForestRegressor,6.413075,0.257073,0.440403
1,XGBRegressor,7.102914,0.332713,0.313539
0,CatBoostRegressor,7.263349,0.309797,0.282178
2,LGBMRegressor,7.394393,0.419739,0.256043


✅ 最佳回归模型：RandomForestRegressor | RMSE=6.413075, R²=0.4404


## Chunk 6 — 特征贡献度（SHAP 或特征重要性）


In [7]:
# === Chunk 6: Explainability ===
if target_reg is not None and 'best_reg' in locals():
    pipe = best_reg["pipe"]
    try:
        final_model = pipe.named_steps["mdl"]
        ohe = pipe.named_steps["prep"].named_transformers_["cat"]
        feature_names = list(ohe.get_feature_names_out(cat_cols)) + num_cols

        if SHAP_OK:
            try:
                sample_X = pipe.named_steps["prep"].transform(Xte)[:300]
                import shap
                explainer = shap.Explainer(final_model.predict, sample_X)
                shap_values = explainer(sample_X[:100])
                print("SHAP 全局重要性（绝对平均值 Top10）：")
                imp = np.abs(shap_values.values).mean(axis=0)
                top_idx = np.argsort(imp)[::-1][:10]
                for i in top_idx:
                    print(f"  - {feature_names[i]}: {imp[i]:.6f}")
            except Exception as e:
                print("⚠️ SHAP 计算失败，退化到 feature_importances_：", e)
                raise
        else:
            raise RuntimeError("SHAP 不可用")

    except Exception as e:
        try:
            importances = final_model.feature_importances_
            order = np.argsort(importances)[::-1][:10]
            print("特征重要性 Top10（基于树模型）：")
            for i in order:
                print(f"  - {feature_names[i]}: {importances[i]:.6f}")
        except Exception as e2:
            print("❌ 无法提取特征重要性：", e2)
else:
    print("⚠️ 无法进行可解释性分析。")

PermutationExplainer explainer: 101it [00:27,  2.45it/s]                        

SHAP 全局重要性（绝对平均值 Top10）：
  - Views: 0.335742
  - Likes: 0.119901
  - Shares: 0.010286
  - Comments: 0.003807
  - Weekday: 0.001109
  - Platform_YouTube: 0.000845
  - Month: 0.000578
  - Region_Canada: 0.000368
  - Content_Type_Reel: 0.000335
  - Hashtag_#Music: 0.000264


## Chunk 7 — 时间趋势分析（Prophet 或季节性分解）


In [8]:
# === Chunk 7: Time trend ===
dtcol = CFG["date_col"]
if dtcol in df.columns and "Engagement_Rate" in df.columns:
    ts = df.dropna(subset=[dtcol]).set_index(dtcol)["Engagement_Rate"].resample("W").mean().dropna()
    print("时间序列点数：", len(ts))

    if PROPHET_OK and len(ts)>20:
        tmp = ts.reset_index().rename(columns={dtcol:"ds","Engagement_Rate":"y"})
        m = Prophet(seasonality_mode="additive", weekly_seasonality=True, daily_seasonality=False)
        m.fit(tmp)
        future = m.make_future_dataframe(periods=8, freq="W")
        fcst = m.predict(future).tail(8)[["ds","yhat","yhat_lower","yhat_upper"]]
        print("未来 8 周预测（平均互动率）：")
        display(fcst)
        print("分析：若 yhat 呈上升趋势，可加码；若不确定性区间变宽，需谨慎预算分配。")
    elif SM_SEASONAL_OK and len(ts)>20:
        from statsmodels.tsa.seasonal import seasonal_decompose
        res = seasonal_decompose(ts, model="additive", period=4)
        print("季节性/趋势分解完成。趋势头部 5：")
        display(res.trend.dropna().head())
        print("分析：观察 trend 与 seasonal，可指导“周内/季度”发帖节奏。")
    else:
        print("⚠️ 数据点不足或依赖不可用，跳过时间序列预测。")
else:
    print("⚠️ 缺少日期列或互动率，跳过时间趋势分析。")

时间序列点数： 105


01:24:21 - cmdstanpy - INFO - Chain [1] start processing
01:24:22 - cmdstanpy - INFO - Chain [1] done processing


未来 8 周预测（平均互动率）：


,ds,yhat,yhat_lower,yhat_upper
105,2024-01-07,0.735176,-0.022815,1.546408
106,2024-01-14,0.738392,-0.087954,1.482836
107,2024-01-21,0.741609,-0.042823,1.525645
108,2024-01-28,0.744826,-0.059672,1.493412
109,2024-02-04,0.748043,-0.000931,1.585894
110,2024-02-11,0.751260,-0.039214,1.469342
111,2024-02-18,0.754477,-0.025112,1.552130
112,2024-02-25,0.757693,-0.004425,1.587108


分析：若 yhat 呈上升趋势，可加码；若不确定性区间变宽，需谨慎预算分配。


## Chunk 8 — 爆款分类（平台内 Top p%，p 来源于 CFG）


In [20]:
# === Chunk 8: Virality classification — 强化 & 静默版 ===
from sklearn.feature_selection import VarianceThreshold
from collections import Counter
from packaging import version
import sklearn

VIRAL_P = CFG["viral_percentile"]

def build_viral_labels(_df, metric="Engagement_Rate", platform_col="Platform", p=VIRAL_P):
    df2 = _df.copy()
    if metric not in df2.columns:
        return None
    df2[metric] = pd.to_numeric(df2[metric], errors="coerce")
    if platform_col in df2.columns:
        def _mark(g):
            thr = g[metric].quantile(p)
            g["viral"] = (g[metric] >= thr).astype(int)
            return g
        return df2.groupby(platform_col, group_keys=False).apply(_mark)
    else:
        thr = df2[metric].quantile(p)
        df2["viral"] = (df2[metric] >= thr).astype(int)
        return df2

if "Engagement_Rate" in df.columns:
    df_v = build_viral_labels(df, metric="Engagement_Rate", platform_col="Platform", p=VIRAL_P)
    if df_v is not None and df_v["viral"].nunique() == 2:
        # ① 合并低频类别（不止 Hashtag）
        def collapse_rare(series, k=20):
            vc = series.astype(str).value_counts()
            keep = set(vc.head(k).index)
            return series.astype(str).where(series.astype(str).isin(keep), "_other_")

        cat_cols_c = [c for c in ["Platform","Content_Type","Region","Hashtag"] if c in df.columns]
        for c in cat_cols_c:
            df_v[c] = collapse_rare(df_v[c], k=20)

        num_cols_c = [c for c in ["Views","Likes","Shares","Comments","Hour","Weekday","Month"] if c in df.columns]

        X_clf = df_v[cat_cols_c + num_cols_c].copy()
        y_clf = df_v["viral"].astype(int).values

        Xtr_c, Xte_c, ytr_c, yte_c = train_test_split(
            X_clf, y_clf,
            test_size=CFG["test_size"],
            random_state=CFG["random_state"],
            stratify=y_clf
        )

        # 类别不平衡权重（给 LGBM / XGB 用）
        cnt = Counter(ytr_c)
        pos = cnt.get(1, 0)
        neg = cnt.get(0, 0)
        scale_pos_weight = float(neg) / max(1.0, float(pos)) if (pos > 0) else 1.0

        # ② 预处理：OHE 合并稀有类别 + 方差过滤近常量列
        supports_minfreq = version.parse(sklearn.__version__) >= version.parse("1.1")
        if supports_minfreq:
            ohe = OneHotEncoder(
                handle_unknown="infrequent_if_exist",
                min_frequency=0.01,   # 1% 以下并入“infrequent”
                sparse=True
            )
        else:
            ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

        pre_clf = ColumnTransformer([
            ("cat", ohe, cat_cols_c),
            ("num", StandardScaler(with_mean=False), num_cols_c)  # 稀疏兼容
        ])
        vt = VarianceThreshold(threshold=1e-5)  # 比 0 更严格，去近常量列

        def build_clf_model(backend: str):
            if backend == "catboost" and CATBOOST_OK:
                params = CFG["catboost"].copy()
                return ("CatBoostClassifier",
                        CatBoostClassifier(random_state=CFG["random_state"],
                                           loss_function="Logloss",
                                           **params))
            if backend == "xgboost" and XGBOOST_OK:
                p = CFG["xgboost"].copy()
                p.update(dict(objective="binary:logistic",
                              eval_metric="logloss",
                              scale_pos_weight=scale_pos_weight,
                              tree_method="hist",
                              n_jobs=-1))
                return ("XGBClassifier", XGBClassifier(random_state=CFG["random_state"], **p))
            if backend == "lightgbm" and LGBM_OK:
                # —— 更保守 + 静默 —— 
                p = dict(
                    n_estimators=CFG["lightgbm"].get("n_estimators", 700),
                    learning_rate=CFG["lightgbm"].get("learning_rate", 0.05),
                    num_leaves=15,                 # ↓小一些更稳
                    max_depth=6,                   # 限深避免盲目下探
                    min_child_samples=5,           # 放宽，提高可分叶
                    min_split_gain=0.0,
                    min_child_weight=1e-3,
                    feature_fraction=1.0,          # 不抽列，避免更少可分列
                    bagging_fraction=1.0,          # 不抽行
                    bagging_freq=0,
                    objective="binary",
                    scale_pos_weight=scale_pos_weight,
                    reg_alpha=0.0,
                    reg_lambda=0.0,
                    n_jobs=-1,
                    random_state=CFG["random_state"],
                    verbosity=-1                   # ← 静默 LightGBM 日志（含你看到的 warning）
                )
                return ("LGBMClassifier", LGBMClassifier(**p))
            if backend == "rf":
                return ("RandomForestClassifier",
                        RandomForestClassifier(random_state=CFG["random_state"],
                                               n_jobs=-1,
                                               **CFG["rf"]))
            return None

        clf_candidates = []
        for b in CFG["clf_backends"]:
            pair = build_clf_model(b)
            if pair is not None:
                clf_candidates.append(pair)

        best_f1 = -1.0
        best_clf = None
        for name, clf in clf_candidates:
            pipe = Pipeline([
                ("prep", pre_clf),
                ("vt", vt),          # 过滤近常量列
                ("clf", clf)
            ])
            pipe.fit(Xtr_c, ytr_c)
            ypred = pipe.predict(Xte_c)
            report = classification_report(yte_c, ypred, digits=4, output_dict=True)
            f1 = report["weighted avg"]["f1-score"]
            print(f"📈 分类模型：{name} | weighted F1={f1:.4f}")
            if f1 > best_f1:
                best_clf = (name, pipe, f1)
                best_f1 = f1

        print(f"✅ 最佳分类模型：{best_clf[0]} | weighted F1={best_clf[2]:.4f}")
        ypred_best = best_clf[1].predict(Xte_c)
        print(classification_report(yte_c, ypred_best, digits=4))
        print("分析：F1 越高表示爆款/非爆款整体表现更好；用 OHE 合并稀有 + 方差过滤可减少“无正增益”情况。")
        print("提示：若依旧出现该提示且你不需要 LGBM，可在 CFG['clf_backends'] 里移除 'lightgbm'，优先用 CatBoost/XGBoost。")
    else:
        print("⚠️ 无法构造二分类标签（可能样本不足或目标列缺失）。")
else:
    print("⚠️ 无法进行爆款分类（缺少 Engagement_Rate）。")

📈 分类模型：CatBoostClassifier | weighted F1=0.9873
📈 分类模型：XGBClassifier | weighted F1=0.9865
📈 分类模型：LGBMClassifier | weighted F1=0.9873
📈 分类模型：RandomForestClassifier | weighted F1=0.9803
✅ 最佳分类模型：LGBMClassifier | weighted F1=0.9873
              precision    recall  f1-score   support

           0     0.9955    0.9902    0.9929      1125
           1     0.9160    0.9600    0.9375       125

    accuracy                         0.9872      1250
   macro avg     0.9558    0.9751    0.9652      1250
weighted avg     0.9876    0.9872    0.9873      1250

分析：F1 越高表示爆款/非爆款整体表现更好；用 OHE 合并稀有 + 方差过滤可减少“无正增益”情况。
提示：若依旧出现该提示且你不需要 LGBM，可在 CFG['clf_backends'] 里移除 'lightgbm'，优先用 CatBoost/XGBoost。


## Chunk 9 — 受众/内容聚类（KMeans）


In [13]:
# === Chunk 9: KMeans clustering ===
cat_cols_k = [c for c in ["Platform","Content_Type","Region","Hashtag"] if c in df.columns]
num_cols_k = [c for c in ["Views","Likes","Shares","Comments","Hour","Weekday","Month","Engagement_Rate"] if c in df.columns]

if len(num_cols_k)+len(cat_cols_k) > 0:
    pre_all = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols_k),
                                 ("num", StandardScaler(), num_cols_k)])
    X_all = pre_all.fit_transform(df[cat_cols_k + num_cols_k].fillna(0))

    best_k, best_score = None, -1
    for k in [3,4,5,6]:
        try:
            km = KMeans(n_clusters=k, random_state=CFG["random_state"], n_init=10)
            labels = km.fit_predict(X_all)
            score = silhouette_score(X_all, labels)
            print(f"k={k} -> 轮廓系数={score:.4f}")
            if score > best_score:
                best_k, best_score, best_km = k, score, km
        except Exception as e:
            print(f"k={k} 计算失败：", e)

    if best_k is not None:
        labels = best_km.predict(X_all)
        df["cluster"] = labels
        print(f"✅ 选定簇数：k={best_k} | 轮廓系数={best_score:.4f}")
        print("各簇规模：")
        display(df["cluster"].value_counts().rename("count").to_frame())

        if num_cols_k:
            print("各簇数值特征均值：")
            display(df.groupby("cluster")[num_cols_k].mean().round(4))
        print("分析：不同簇的特征均值差异，能辅助制定差异化内容/投放策略。")
    else:
        print("⚠️ 聚类未成功。")
else:
    print("⚠️ 无足够特征用于聚类。")

k=3 -> 轮廓系数=0.0768
k=4 -> 轮廓系数=0.0690
k=5 -> 轮廓系数=0.0671
k=6 -> 轮廓系数=0.0667
✅ 选定簇数：k=3 | 轮廓系数=0.0768
各簇规模：


,count
cluster,
0,2515
1,2484
2,1


各簇数值特征均值：


,Views,Likes,Shares,Comments,Hour,Weekday,Month,Engagement_Rate
cluster,,,,,,,,
0,2.577800e+06,248669.0584,54579.0036,24933.1769,0.0,4.6998,6.6771,0.4786
1,2.410291e+06,254288.8961,46429.1067,24838.5298,0.0,1.2834,6.4690,0.5445
2,1.266000e+03,318849.0000,1715.0000,36121.0000,0.0,6.0000,9.0000,281.7417


分析：不同簇的特征均值差异，能辅助制定差异化内容/投放策略。


## Chunk 10 — 广告 ROI 建模（若字段存在）


In [14]:
# === Chunk 10: Ads ROI modeling (optional) ===
ad_cols = [c for c in ["AdSpend","ClickThroughRate","ConversionRate","WebsiteVisits","PagesPerVisit","TimeOnSite","EmailOpens","EmailClicks"] if c in df.columns]

if "AdSpend" in ad_cols and "ConversionRate" in ad_cols:
    df_ads = df.copy()
    df_ads["eCPA"] = (df_ads["AdSpend"] / (df_ads["ConversionRate"].replace(0, np.nan))).replace([np.inf, -np.inf], np.nan)
    df_ads = df_ads.dropna(subset=["eCPA"])

    cat_ad = [c for c in ["Platform","Content_Type","Region"] if c in df_ads.columns]
    num_ad = [c for c in ["Views","Likes","Shares","Comments","Hour","Weekday","Month","AdSpend","ClickThroughRate","ConversionRate"] if c in df_ads.columns]

    X_ad = df_ads[cat_ad + num_ad].copy()
    y_ad = df_ads["eCPA"].astype(float).values

    pre_ad = ColumnTransformer([("cat", OneHotEncoder(handle_unknown="ignore"), cat_ad),
                                ("num", StandardScaler(), num_ad)])

    # 用统一配置选择模型
    def build_reg_model(backend: str):
        if backend == "catboost" and CATBOOST_OK:
            return ("CatBoostRegressor", CatBoostRegressor(random_state=CFG["random_state"], **CFG["catboost"]))
        if backend == "xgboost" and XGBOOST_OK:
            return ("XGBRegressor", XGBRegressor(random_state=CFG["random_state"], **CFG["xgboost"], objective="reg:squarederror"))
        if backend == "lightgbm" and LGBM_OK:
            return ("LGBMRegressor", LGBMRegressor(random_state=CFG["random_state"], **CFG["lightgbm"]))
        if backend == "rf":
            return ("RandomForestRegressor", RandomForestRegressor(random_state=CFG["random_state"], n_jobs=-1, **CFG["rf"]))
        return None

    pair = None
    for b in CFG["reg_backends"]:
        pair = build_reg_model(b)
        if pair is not None:
            break

    mdl_name, mdl = pair
    pipe_ad = Pipeline([("prep", pre_ad), ("mdl", mdl)])
    Xtr_a, Xte_a, ytr_a, yte_a = train_test_split(X_ad, y_ad, test_size=CFG["test_size"], random_state=CFG["random_state"])
    pipe_ad.fit(Xtr_a, ytr_a)
    pred_a = pipe_ad.predict(Xte_a)

    rmse = mean_squared_error(yte_a, pred_a, squared=False)
    mae  = mean_absolute_error(yte_a, pred_a)
    r2   = r2_score(yte_a, pred_a)

    print(f"📊 ROI 模型（{mdl_name}，eCPA 预测）")
    print(f"  RMSE={rmse:.4f}  MAE={mae:.4f}  R²={r2:.4f}")
    print("分析：RMSE/MAE 越小越好，R² 越接近 1 越好；可辅助预算分配与渠道优先级。")
else:
    print("⚠️ 未检测到广告字段（AdSpend/CTR/ConversionRate 等）。")
    print("建议：如缺乏广告数据，可用 Engagement_Rate 作为内容效能的替代指标，")
    print("并在媒体平台报表中结合曝光/点击/转化来补齐 ROI 视角。")

⚠️ 未检测到广告字段（AdSpend/CTR/ConversionRate 等）。
建议：如缺乏广告数据，可用 Engagement_Rate 作为内容效能的替代指标，
并在媒体平台报表中结合曝光/点击/转化来补齐 ROI 视角。


## Chunk 11 — 敏感性分析（What-if 模拟）


In [15]:
# === Chunk 11: Sensitivity analysis ===
def what_if_change(pipe, df_src, feature, values, cat_cols, num_cols, target_name="Engagement_Rate"):
    df_tmp = df_src[cat_cols + num_cols].copy()
    outs = []
    if feature not in df_tmp.columns:
        return None
    baseline = pipe.predict(df_tmp.head(200))
    for v in values:
        df_changed = df_tmp.copy()
        df_changed[feature] = v
        pred = pipe.predict(df_changed.head(200))
        outs.append((v, float(np.mean(pred - baseline))))
    return outs

if target_reg is not None and 'best_reg' in locals():
    pipe = best_reg["pipe"]
    candidates = [c for c in ["Hour","Weekday","Views"] if c in df.columns]
    sens_all = []
    for f in candidates:
        vals = sorted(df[f].dropna().unique())
        import numpy as np
        if len(vals)>10:
            vals = np.linspace(np.nanmin(vals), np.nanmax(vals), 8).astype(int if df[f].dtype.kind in "iu" else float)
        diffs = what_if_change(pipe, df, f, vals, cat_cols, num_cols, target_name="Engagement_Rate")
        if diffs:
            mean_abs = np.mean([abs(x[1]) for x in diffs])
            sens_all.append((f, mean_abs, diffs))

    if sens_all:
        sens_all.sort(key=lambda x: x[1], reverse=True)
        print("敏感性排名（按平均影响绝对值）：")
        for f, mag, _ in sens_all:
            print(f"  - {f}: {mag:.6f}")
        top = sens_all[0]
        print("\n示例 —— 最敏感特征的变化曲线：", top[0])
        for v, d in top[2]:
            print(f"  {top[0]}={v}  ->  预测变化(相对基线均值)={d:.6f}")
    else:
        print("⚠️ 无法计算敏感性（可能样本不足）。")
else:
    print("⚠️ 无法进行敏感性分析。")

敏感性排名（按平均影响绝对值）：
  - Views: 7.600751
  - Weekday: 0.014317
  - Hour: 0.000000

示例 —— 最敏感特征的变化曲线： Views
  Views=1266  ->  预测变化(相对基线均值)=57.983478
  Views=715289  ->  预测变化(相对基线均值)=-0.135649
  Views=1429312  ->  预测变化(相对基线均值)=-0.349206
  Views=2143336  ->  预测变化(相对基线均值)=-0.418870
  Views=2857359  ->  预测变化(相对基线均值)=-0.455016
  Views=3571383  ->  预测变化(相对基线均值)=-0.475819
  Views=4285406  ->  预测变化(相对基线均值)=-0.490866
  Views=4999430  ->  预测变化(相对基线均值)=-0.497104


## Chunk 12 — 残差与误差结构


In [17]:
# === Chunk 12: Residuals ===
if target_reg is not None and 'best_reg' in locals():
    pipe = best_reg["pipe"]
    pred_full = pipe.predict(df[cat_cols + num_cols].copy())
    residuals = df["Engagement_Rate"].values - pred_full
    import numpy as np
    print("残差均值：", float(np.mean(residuals)))
    print("残差标准差：", float(np.std(residuals)))
    print("过大异常（>|3σ|）占比：", float(np.mean(np.abs(residuals) > (3*np.std(residuals)))))
    print("分析：残差均值接近 0 且极端残差占比低，说明模型总体无系统性偏差。")
else:
    print("⚠️ 残差分析不可用。")

残差均值： 0.049727079833028785
残差标准差： 3.233105426521446
过大异常（>|3σ|）占比： 0.001
分析：残差均值接近 0 且极端残差占比低，说明模型总体无系统性偏差。


## Chunk 13 — 行业化总结与策略建议（A+B+C）


In [18]:
# === Chunk 13: Summary ===
print("—— 总结报告（自动生成） ——")
if 'best_reg' in locals():
    print(f"[内容预测] 最佳回归模型：{best_reg['model']}，RMSE={best_reg['RMSE']:.6f}，R²={best_reg['R2']:.4f}。")
    print("解读：该模型对互动率的解释力较强，可用于内容选题与投放排期的量化评估。")
else:
    print("[内容预测] 数据或依赖不足，未完成回归建模。")

if 'cluster' in df.columns:
    vc = df['cluster'].value_counts()
    print(f"[受众/内容聚类] 最优簇数：{len(vc)}，簇规模：{list(vc.values)}")
    print("解读：结合各簇的数值均值差异，可为不同人群输出差异化内容与投放时段建议。")
else:
    print("[受众/内容聚类] 暂无聚类结果。")

if "AdSpend" in df.columns and "ConversionRate" in df.columns:
    print("[广告ROI] 已完成 eCPA 预测，可用于不同渠道/创意的预算分配。")
else:
    print("[广告ROI] 数据缺少投放字段，建议在媒体端补充曝光、点击、转化数据后再建模。")

print("—— 策略建议 ——")
print("1) 平台差异：高基线平台可追求规模，低基线平台追求ROI；结合时间段（Hour/Weekday）的敏感性结果进行排期优化。")
print("2) 内容类型：优先制作对互动率贡献度高的内容类型；必要时做 A/B 测试验证。")
print("3) 广告投放：若可用，使用 ROI 模型对 eCPA 进行预测分层，预算向低 eCPA、稳定人群倾斜。")
print("4) 受众分群：对每个簇制定不同创意与投放时段；将聚类标签回填至BI系统持续跟踪表现。")


—— 总结报告（自动生成） ——
[内容预测] 最佳回归模型：RandomForestRegressor，RMSE=6.413075，R²=0.4404。
解读：该模型对互动率的解释力较强，可用于内容选题与投放排期的量化评估。
[受众/内容聚类] 最优簇数：3，簇规模：[2515, 2484, 1]
解读：结合各簇的数值均值差异，可为不同人群输出差异化内容与投放时段建议。
[广告ROI] 数据缺少投放字段，建议在媒体端补充曝光、点击、转化数据后再建模。
—— 策略建议 ——
1) 平台差异：高基线平台可追求规模，低基线平台追求ROI；结合时间段（Hour/Weekday）的敏感性结果进行排期优化。
2) 内容类型：优先制作对互动率贡献度高的内容类型；必要时做 A/B 测试验证。
3) 广告投放：若可用，使用 ROI 模型对 eCPA 进行预测分层，预算向低 eCPA、稳定人群倾斜。
4) 受众分群：对每个簇制定不同创意与投放时段；将聚类标签回填至BI系统持续跟踪表现。
